<a href="https://colab.research.google.com/github/anubhavgupta1/Dive-Into-Deep-Learning/blob/main/Deep%20Learning%20Computation/GPU%20Management/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Deep Learning Computation

In [1]:
!nvidia-smi

Tue Jan 26 13:19:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf

### Computing Devices

In [3]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f1d8d3b0448>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f1d8d3b0508>)

In [4]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

In [5]:
def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

In [6]:
try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f1d8d3b7108>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7f1d8d3a8788>])

In [7]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

### Storage on the GPU

In [8]:
import time

In [9]:
%%time
with try_gpu():
    X = tf.ones((10000, 10000))
    Y = tf.random.uniform((10000, 10000))

CPU times: user 3.85 ms, sys: 20.1 ms, total: 23.9 ms
Wall time: 41.9 ms


### Neural Networks and GPUs

In [10]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [11]:
net(X)

<tf.Tensor: shape=(10000, 1), dtype=float32, numpy=
array([[1.6568325],
       [1.6568325],
       [1.6568325],
       ...,
       [1.6568325],
       [1.6568325],
       [1.6568325]], dtype=float32)>

In [12]:
## Let us confirm that the model parameters are stored on the same GPU.
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')